# 랜덤 포레스트

## 랜덤 포레스트 배경 - 앙상블
- 여러 Base 모델들의 예측을 다수결 법칙 또는 평균을 이용해 통합하여 예측 정확성을 향상시키는 방법
- 다음 조건을 만족할 때 앙상블 모델은 Base 모델보다 우수한 성능을 보여줌
 - Base 모델들이 서로 독립적
 - Base 모델들이 무작위 예측을 수행하는 모델보다 성능이 좋은 경우(최소한의 성능보다 좋게 나와야하는 것 이진분류에서는 0.5)

- 의사결정나무모델은 앙상블 모델의 base 모델로써 활용도가 높음
 - Low computational complexity: 데이터의 크기가 방대한 경우에도 모델을 빨리 구축할 수 있음
 - Nonparametric(비모수적 모델): 데이터 분포에 대한 전제가 필요하지 않음

## 랜덤 포레스트 개요
- 다수의 의사결정나무모델에 의한 예측을 종합하는 앙상블 방법
- 일반적으로 하나의 의사결정나무모델 보다 높은 예측 정확성을 보여줌
- 관측치 수에 비해 변수의 수가 많은 고차원 데이터에서 중요 변수 선택 기법으로 널리 활용

![대체 텍스트](https://www.spotx.tv/wp-content/uploads/Figure-1-Random-Forest-Decision-Tree.png)

1. Bootstrap 기법을 이용하여 다수의 training data 생성
2. 생성된 training data로 decision tree 모델 구축(무작위 변수를 사용하여)
3. 예측 종합

- 핵심 아이디어: Diversity, Random 확보
> 1. 여러 개의 Training data를 생성하여 각 데이터마다 개별 의사결정나무모델 구축 -> **Bagging★**
> 2. 의사결정나무모델 구축 시 변수 무작위로 선택->**Random subspace★**

### Bagging(Bootstrap Aggregating)
Input data가 연속형: average, 이산형(범주형): voting

- Bootstrapping -> sampling
 - 각 모델은 **서로 다른 학습 데이터셋**을 이용
 - 각 데이터셋은 복원추출(sampling with replacement)을 통해 **원래 데이터의 수만큼의 크기를 갖도록 샘플링**
 - 개별 데이터셋을 붓스트랩셋이라고 부름

 ![대체 텍스트](https://bradleyboehmke.github.io/HOML/images/bootstrap-scheme.png)

- Result Aggregating
 
 ![대체 텍스트](https://seokchaeyoon.github.io/images/test.PNG)
 
 - majority voting
  >> 1:6개 0:4개 -> 1로 반환

 - Weighted voting(traing accuracy of individual models)
  >> 0: 0.91 + 0.77 + 0.95 + 0.82 / 전체 모델의 Accuracy
  
   >> 1: 0.8 + 0.75 +0.88 +0.65 + 0.78 + 0.83 / 전체 모델의 Accuracy

 - Weighted voting(predicted probablity for each class)
  >> 1: 0.9 + 0.92 + 0.87 +..+ 0.57 / 10 
  
   >> 0:  (1-0.9) + (1-0.92) +...+(1-0.57) /10 



## Random subspace
- 의사결정 나무의 분기점을 탐색할 떄, **원래 변수의 수보다 적은 수의 변수를 임의로 선택**하여 해당 변수들만을 고려 대상으로함

![대체 텍스트](https://www.python-course.eu/images/Bagging_Subspace_sampling.png)


## 랜덤 포레스트 - Generalization Error
- 각각의 개별 tree는 과적합 될 수 있음
- Random forest는 tree수가 충분히 많을 때 Strong Law of Large Numbers에 의해 과적합되지 않고 그 에러는 limiting value에 수렴됨
 > **Generalization error <= p(1-s^2)/s^2** ★
  - p: Decision tree 사이의 평균 상관관계
  - s: 올바로 예측한 tree와 잘못 예측한 tree수 차이의 평균
- 개별 tree의 정확도가 높을수록 s 증가
- Bagging과 random subspace 기법은 각 모델들의 독립성, 일반화, 무작위성을 최대화 시켜 모델간의 상관관계 p를 감소시킴
- 개별 Tree의 정확도, 독립성이 높을수록 random forest의 성능이 높아짐

## 랜덤 포레스트 - 중요변수 선택
- 변수중요도
 - 랜덤 포레스트는 선형 회귀모델/로지스틱 회귀모델과는 달리 개별 변수가 통계적으로 얼마나 유의한지에 대한 정보를 제공하지 않음
 - 대신 랜덤 포레스트는 다음과 같은 간접적인 방식으로 변수의 중요도를 결정
 > 1단계: 원래 데이터 집합에 대해서 Out of bag(OOB)Error를 구함 (ri, i=1,2,3..,t)
 
 >> 2단계: 특정 변수의 값을 임의로 뒤섞은 데이터 집합에 대해서 OOB Error를 구함(ei, i=1,2,3..,t)

 >> 3단계: 개별 변수의 중요도는 2단계와 1단계 OOB Error 차이의 평균과 분산을 고려하여 결정 (di = |ri - ei|)
   >>> - 특정변수의 변수 중요도: mean(d)/s(d)

- Out of bag(OOB)
 - 배깅을 사용할 경우 붓스트랩셋에 포함되지 않은 데이터들을 검증 집합으로 사용함

 ![대체 텍스트](https://rasbt.github.io/mlxtend/user_guide/evaluate/BootstrapOutOfBag_files/bootstrap_concept.png) 

## 랜덤 포레스트 - 하이퍼 파라미터
1. Decision tree의 수
- Strong law of lange numbers을 만족 시키기 위해 2,000개 이상의 decision tree 필요
2. Decision tree에서 노드 분할 시 무작위로 선택되는 변수의 수
- 일반적으로 변수의 수에 따라 다음과 같이 추천됨
- Classification: 제곱근(변수의수)
- Regreesion: 변수의 수/3